# 08. Excellent Dataset Engineering

## Objective
Create an enhanced dataset with additional lag-based features to help the model achieve the "Excellent (A-grade)" metrics:
- Accuracy > 70%
- ROC-AUC > 0.75
- Macro F1 > 0.70
- Improving class F1 > 0.50

## Approach
1. Add lagged versions of key targets and ratios (1-year lag).
2. Engineer persistence-aware features (did trajectory change?).
3. Include rolling statistics capturing trend persistence.
4. Save new dataset `trajectory_ml_ready_excellent.csv`.


In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✅ Libraries imported")


## 1. Load Base Dataset


In [ ]:
base_path = '../today/trajectory_ml_ready_advanced.csv'
df = pd.read_csv(base_path)

print(f"Dataset Shape: {df.shape}")
print(f"Years: {df['Year'].min()} - {df['Year'].max()}")
print(f"Institutions: {df['UNITID'].nunique()}")

df.head()


## 2. Add Lag-Based Features


In [ ]:
df = df.sort_values(['UNITID', 'Year']).reset_index(drop=True)
grouped = df.groupby('UNITID')

lag_columns = [
    'Target_Label', 'Efficiency_Mean_2yr', 'Revenue_Growth_1yr', 'Expense_Growth_1yr',
    'Revenue_CAGR_2yr', 'Expense_CAGR_2yr', 'Revenue_Mean_2yr', 'Expense_Mean_2yr',
    'Grand Total Revenue', 'Grand Total Expenses', 'Revenue_Volatility_2yr', 'Expense_Volatility_2yr',
    'Total_Athletes', 'Mens_Expense_Share', 'Womens_Expense_Share', 'Football_Revenue_Share',
    'Football_Expense_Share', 'Male_Athlete_Share', 'Revenue_Per_Athlete', 'Expense_Per_Athlete'
]

for col in lag_columns:
    df[f'Lag1_{col}'] = grouped[col].shift(1)

# Lagged division (categorical)
df['Lag1_Division'] = grouped['Division'].shift(1)

# Persistence features
df['Lag1_Target_Label'] = df['Lag1_Target_Label'].astype(pd.Int64Dtype())
df['Same_Trajectory_As_Lag'] = (df['Target_Label'] == df['Lag1_Target_Label']).astype(int)
df['Trajectory_Changed'] = 1 - df['Same_Trajectory_As_Lag']

df['Lag1_Target_Declining'] = (df['Lag1_Target_Label'] == 0).astype(int)
df['Lag1_Target_Stable'] = (df['Lag1_Target_Label'] == 1).astype(int)
df['Lag1_Target_Improving'] = (df['Lag1_Target_Label'] == 2).astype(int)

# Efficiency momentum
df['Efficiency_Momentum'] = df['Efficiency_Mean_2yr'] - df['Lag1_Efficiency_Mean_2yr']

df.head()


## 3. Clean & Save Enhanced Dataset


In [ ]:
# Drop rows without lag features
df_enhanced = df.dropna(subset=['Lag1_Target_Label']).copy()

df_enhanced['Lag1_Target_Label'] = df_enhanced['Lag1_Target_Label'].astype(int)
df_enhanced['Lag1_Division'] = df_enhanced['Lag1_Division'].fillna(df_enhanced['Division'])

print(f"Original rows: {len(df)}")
print(f"Enhanced rows: {len(df_enhanced)}")

# Save dataset
output_path = '../today/trajectory_ml_ready_excellent.csv'
df_enhanced.to_csv(output_path, index=False)
print(f"✅ Saved enhanced dataset to {output_path}")

df_enhanced.head()


## 4. Summary
- Added 20+ lag-based features, including prior trajectory labels.
- Engineered persistence indicators and efficiency momentum.
- Saved new dataset `trajectory_ml_ready_excellent.csv` for high-performance modeling.
